In [1]:
import requests
import lxml
from bs4 import BeautifulSoup
import time
import csv
import re
import random
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.common.exceptions import NoSuchElementException
import urllib3
urllib3.disable_warnings()

In [ ]:
#from bs4 import builder
#from bs4.builder import _lxml
#builder.register_treebuilders_from(_lxml)

In [ ]:
proxies = {'http': 'http://cnproxy.int.nokia-sbell.com/proxy.pac', 'https': 'http://cnproxy.int.nokia-sbell.com/proxy.pac'}
filter_out = re.compile(u'(药|减肥|瘦身|手机壳|(手机|钢化)膜|手机卡|手机号|(充电|数据|转接)线)|情趣|避孕|婴儿|(狗|猫|龟|狐|鼠)|保健品|滋补|成人用品|玻尿酸|睡衣|家居服|内裤|文胸|T恤|打底|自慰|飞机杯|课程|养生茶|题库')
filter_in = re.compile(u'(杯|酒|壶|茶|玉|玛瑙|灯|桌|咖啡|笔记本|塑料袋|[酱|调]料|文玩|水培|笔|[U|优]盘|电脑|主机|显示器|摄像头|乐器|背包|空调|洗衣机|冰箱|手环|风扇|电视|玩具|书籍|办公用品|得力|洗衣机|净化器|螺丝刀|机器人|伞|美的|阿道夫|挂烫机|度计|飞科|沃隆|按摩|惠普|零食|音)')
#pattern  = re.compile(usr_condition)

url = '''https://try.jd.com/activity/getActivityList?page=%s'''
try_items={}
for i in range(1,100):
    wb_data =requests.get(url%i,proxies=proxies, verify=False)
    print("Working on %s"%(url%i))
    time.sleep(2)
    soup = BeautifulSoup(wb_data.text,'lxml')
    for try_item in soup.find_all("div",class_='try-item'):
        name = try_item.findChild('div',class_='p-name').string
        if (name != None) and (filter_out.search(name) == None):
            if filter_in.search(name) != None:
                try_items[name]='https:'+try_item.findChild('a',class_='link')['href']

Working on https://try.jd.com/activity/getActivityList?page=1
Working on https://try.jd.com/activity/getActivityList?page=2
Working on https://try.jd.com/activity/getActivityList?page=3
Working on https://try.jd.com/activity/getActivityList?page=4
Working on https://try.jd.com/activity/getActivityList?page=5
Working on https://try.jd.com/activity/getActivityList?page=6
Working on https://try.jd.com/activity/getActivityList?page=7
Working on https://try.jd.com/activity/getActivityList?page=8
Working on https://try.jd.com/activity/getActivityList?page=9
Working on https://try.jd.com/activity/getActivityList?page=10
Working on https://try.jd.com/activity/getActivityList?page=11
Working on https://try.jd.com/activity/getActivityList?page=12
Working on https://try.jd.com/activity/getActivityList?page=13
Working on https://try.jd.com/activity/getActivityList?page=14
Working on https://try.jd.com/activity/getActivityList?page=15
Working on https://try.jd.com/activity/getActivityList?page=16
W

In [8]:
prox = Proxy()
#prox.proxy_type = ProxyType.MANUAL
prox.proxy_type = ProxyType.PAC
prox.proxyAutoconfigUrl = 'http://cnproxy.int.nokia-sbell.com/proxy.pac'
#prox.socks_proxy = "135.245.48.34:8000"
#prox.ssl_proxy = "135.245.48.34:8000"

capabilities = webdriver.DesiredCapabilities.CHROME
prox.add_to_capabilities(capabilities)

browser = webdriver.Chrome(desired_capabilities=capabilities)

In [9]:
browser.get( 'https://try.jd.com/activity/getActivityList')
print("Please login your JD account")

Please login your JD account


In [6]:
pass_link=[]
opeation_failed=[]

In [ ]:
count = 0;
new_applied_count = 0
for name in list(try_items.keys()):
    link = try_items[name]
    if link == None or link in pass_link:
        continue
    browser.get(link)
    time.sleep(int(random.random()*10)+6)
    count = count + 1
    if count > 350:
        break
    #allCookies = browser.get_cookies()
    #script = js1#+' var ret = window.fingerprint;return ret;'
    print("-"*20)
    print("Number %d"%count)
    try:
        offer = None
        applied= None
        price = None
        stat = ""
        
        try:
            detail = browser.find_element_by_class_name('p-detail')
            offer = detail.find_element_by_xpath(".//b").text
            applied = detail.find_element_by_xpath(".//b[2]").text
        except NoSuchElementException as e:
            print("无法找到详细信息")
                
        try:
            price = browser.find_element_by_class_name('price').find_element_by_xpath(".//b").text
        except NoSuchElementException as e:
            print("无法找到价格")
        
        try:
            stat = browser.find_element_by_class_name('state').text
        except NoSuchElementException as e:
            print("无法找到申请状态")
        
        print("Offered %s\tApplied %s\tPrice %s\tState %s"%(offer,applied,price,stat))
        if stat.find(u'已提交') > -1 :
            print("已申请该产品：%s\t%s"%(name,link))
        elif stat.find(u'已发布') > -1 :
            print("名单已发布该产品：%s\t%s"%(name,link))
        elif stat.find(u'很遗憾') > -1 :
            print("未获得试用该产品：%s\t%s"%(name,link))
        elif stat.find(u'恭喜') > -1 :
            print("已获取该产品：%s\t%s \t😃"%(name,link))
        else:
            try:
                apply_try = browser.find_element_by_class_name('btn-application')
                apply_try.click()
                time.sleep(5+int(random.random()*10))
            except NoSuchElementException as e:
                #pass
                print("No application")
                #print("Exception on Applying %s\t%s"%(name,link))
            #
            try:
                follow_div = browser.find_element_by_class_name('ui-dialog').find_element_by_class_name('follow')
                follow_link = follow_div.find_element_by_class_name('y')
                follow_link.click()
                time.sleep(5+int(random.random()*10))
            except NoSuchElementException as e:
                #pass
                print("No need follow")
                #print("Exception on Following %s\t%s, maybe followed before."%(name,link))
            
            time.sleep(2+int(random.random()*10))
            try:
                operation_ret = browser.find_element_by_class_name('tip-tit').text
            except NoSuchElementException as e:
                operation_ret = "申请结果未知"
                opeation_failed.append(link)
                
            ret = browser.execute_script(js1)
            print("JS ret = %s"%ret)
            #print(allCookies)
            print("Applied：%s\t%s\t%s"%(name,link,operation_ret))
            if operation_ret.find(u'成功') > -1:
                new_applied_count = new_applied_count + 1
    finally:
        try_items.pop(name)
        print("-"*20)
#browser.close()
print("Done! New applied %s items"%new_applied_count)

--------------------
Number 1
Offered 8	Applied 3356	Price 119.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：南疆巴郎 海盐开心果炭烧腰果碧根果坚果炒货休闲零食每日零食罐装 夏威夷果188g*1	https://try.jd.com/706962.html	申请成功！
--------------------
--------------------
Number 2
Offered 6	Applied 980	Price 31.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：【九润源】洋河 King酒 浓香型白酒 蓝瓶小酒 52度 125ml 单瓶简装	https://try.jd.com/705613.html	申请成功！
--------------------
--------------------
Number 3
Offered 1	Applied 722	Price 39.90	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：茗花有主 桂圆红枣枸杞茶暖暖茶	https://try.jd.com/706204.html	申请成功！
--------------------
--------------------
Number 4
Offered 1	Applied 1820	Price 199.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：探普号2019年易武国有林天门山大树茶普洱茶357g生茶饼	https://try.jd.com/707712.html	申请成功！
--------------------
--------------------
Number 5
Offered 2	Applied 952	Price 15.50	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：艺嘉城文玩核桃青皮四座楼白狮子头满天星麒麟纹官帽公子帽磨盘闷尖南疆石 公子帽青皮53-55mm一对	https://try.jd.com/704878.html	申请成功！
--------------------
------------

Offered 1	Applied 2527	Price 399.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：诗婷露雅玫瑰护肤套装礼盒 补水滋润保湿水乳液霜化妆品套装焕亮肤色细致毛孔清爽舒缓肌肤护肤品旗舰店 礼盒5件套（送保温杯）	https://try.jd.com/705302.html	申请成功！
--------------------
--------------------
Number 40
Offered 1	Applied 989	Price 37.50	State 活动已开始，请快快申请吧！
No need follow
JS ret = 0
Applied：[第2件0元]珍茶集 薄荷叶茶干叶子清火凉茶降脂胆固醇熬夜茶下火可搭配玫瑰花菊花茶茶花草茶50g罐装	https://try.jd.com/707585.html	申请成功！
--------------------
--------------------
Number 41
Offered 20	Applied 2217	Price 75.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：佰草汇 茶 枸杞菊花茶下火茶 菊花茶 安徽菊干枸杞清火茶叶大麦茶决明子甘草降火养生组合花草茶240克 枸杞菊花茶240克	https://try.jd.com/705800.html	申请成功！
--------------------
--------------------
Number 42
Offered 3	Applied 409	Price 17.80	State 活动已开始，请快快申请吧！
No need follow
JS ret = 0
Applied：小麦苗种植盘种子育苗盘 穴盘豆苗水培蔬菜家用无土栽培自制发豆芽盆豆芽菜育苗栽培器 1套加密盘+1包种子+育苗纸+喷壶	https://try.jd.com/704461.html	申请成功！
--------------------
--------------------
Number 43
Offered 2	Applied 1003	Price 28.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：中茶官方旗舰店 云南普洱

No need follow
JS ret = 0
Applied：【少年派同款】劲酒 毛铺 毛铺苦荞酒 45度 毛铺苦荞酒紫荞 500ml 单瓶礼盒	https://try.jd.com/705177.html	申请成功！
--------------------
--------------------
Number 78
Offered 6	Applied 471	Price 29.00	State 活动已开始，请快快申请吧！
No need follow
JS ret = 0
Applied：【九润源】洋河 King酒 浓香型白酒 蓝瓶小酒 42度 125ml 单瓶简装	https://try.jd.com/705615.html	申请成功！
--------------------
--------------------
Number 79
Offered 3	Applied 1621	Price 89.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：龙寅 紫砂壶茶宠摆件段泥一生有钱花生茶具配件 一生有钱	https://try.jd.com/707581.html	申请成功！
--------------------
--------------------
Number 80
Offered 1	Applied 4721	Price 99.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：JBL T110 入耳式通话音乐耳机 手机耳塞 苹果安卓通用 线控耳麦 重低音 游戏耳机 黑色	https://try.jd.com/704474.html	申请成功！
--------------------
--------------------
Number 81
Offered 1	Applied 4166	Price 158.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：五粮液股份公司 52度 浓香型高度白酒 聖酒 圣酒柔和 500ml 一瓶	https://try.jd.com/705996.html	申请成功！
--------------------
--------------------
Number 82
Offered 5	Appli

Offered 2	Applied 2379	Price 79.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：优皓康（ultraDEX）漱口水去口臭牙结石杀菌 口气清新剂 改善牙龈出血 便携式口腔喷雾 无酒精 15ml*10条便携式漱口水（原味）	https://try.jd.com/704640.html	申请成功！
--------------------
--------------------
Number 117
Offered 1	Applied 3530	Price 99.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：御牌 西湖龙井 茶叶 绿茶 正宗梅家坞老茶树 明前特级SSS 龙井茶2019新茶 西门吹雪罐装50g	https://try.jd.com/706344.html	申请成功！
--------------------
--------------------
Number 118
Offered 1	Applied 1515	Price 84.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：水轻轻 柠檬草莓净水美肤韩国花洒过滤器除氯家用沐浴净化器软水淋浴喷头 柠檬香芬维C	https://try.jd.com/705893.html	申请成功！
--------------------
--------------------
Number 119
Offered 3	Applied 5595	Price 138.80	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：公爵（DUKE）LATIN设计师系列钢笔/特细钢笔/宝珠笔明尖笔暗尖笔签字笔 墨水笔芯礼盒套装对笔套 宝珠+钢笔套（0.5mm）时尚蓝	https://try.jd.com/704040.html	申请成功！
--------------------
--------------------
Number 120
Offered 1	Applied 825	Price 39.90	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：老农会 茶叶 花草茶 菊花茶 金丝皇菊一朵一杯散装罐装茶婺源皇菊花茶去火下火茶叶

Offered 1	Applied 863	Price 39.90	State 活动已开始，请快快申请吧！
No need follow
JS ret = 0
Applied：茗花有主 罗汉果恋绿茶15袋红枣枸杞子甘草桐乡杭白菊花养生润喉茉莉绿茶叶独立小袋子包装泡水喝的	https://try.jd.com/706207.html	申请成功！
--------------------
--------------------
Number 155
Offered 1	Applied 1821	Price 99.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：舒理他1000亿plus益生菌发酵低温酸奶无蔗糖无添加 120g*12杯/箱	https://try.jd.com/707050.html	申请成功！
--------------------
--------------------
Number 156
Offered 1	Applied 2523	Price 158.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：百事泰（BESTEK）200W逆变器一拖二USB车载充电器 车充点烟器 12v转220v应急电源 咖啡色	https://try.jd.com/704359.html	申请成功！
--------------------
--------------------
Number 157
Offered 3	Applied 2740	Price 98.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：逗趣家 双层玻璃保温杯男女 透明水杯茶杯钢化玻璃杯子泡茶杯 凉水壶凉水杯茶水杯大容量 蓝色水晶底座	https://try.jd.com/707822.html	申请成功！
--------------------
--------------------
Number 158
Offered 5	Applied 6287	Price 228.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：塔牌绍兴黄酒红牌五年陈花雕酒600ml*6瓶整箱 出口型黄酒手工糯米加饭酒	https://try.jd.com/704027.

JS ret = 0
Applied：晨铭轩  白玉髓四季豆吊坠 男女款玉石挂件项坠玉坠子白玉髓平安扣吊坠男女款送礼物情侣弥勒佛观音项链挂 白色水滴吊坠	https://try.jd.com/704805.html	申请成功！
--------------------
--------------------
Number 193
Offered 1	Applied 517	Price 49.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：氨基酸洁面慕斯 清肌按摩补水去死皮去角质控油抗痘补水保湿洗面奶男女通用温和不刺激收缩毛孔清洁带刷头 #氨基酸洁面慕斯	https://try.jd.com/705606.html	申请成功！
--------------------
--------------------
Number 194
Offered 5	Applied 2555	Price 199.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：无人机航拍飞行器遥控飞机高清专业光流智能定高充电四轴飞行器男孩儿童玩具航模便携可折叠720P高清 白色 115mm无摄像头 买一台发两台	https://try.jd.com/706633.html	申请成功！
--------------------
--------------------
Number 195
Offered 3	Applied 888	Price 15.80	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：帕拉蒂托（PLAYADITO）马黛奶茶 阿根廷马黛茶 男士玄米杯装奶茶休闲饮品	https://try.jd.com/705001.html	申请成功！
--------------------
--------------------
Number 196
Offered 2	Applied 564	Price 49.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：M&U妙媚 魅瞳瞬盈眼线液 初学者 软头无刺激 持久防水不晕染 一笔顺滑 GMPC 植物彩妆欧盟标准	https://try.jd.com/707133.html	申请成功！
-------------

JS ret = 0
Applied：赛尔浦(SAIELRPU)12件家用工具组套螺丝刀内六角美工刀测电笔小工具箱GJ001 GJ001	https://try.jd.com/704743.html	申请成功！
--------------------
--------------------
Number 231
Offered 1	Applied 581	Price 109.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：西婕 身体按摩精油全身夫妻情侣精油按摩润滑精油私处男女按摩油刮痧推油开背 浪漫按摩精油	https://try.jd.com/707993.html	申请成功！
--------------------
--------------------
Number 232
Offered 1	Applied 97	Price 35.00	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：七夕节礼物送女生送男生 生日礼物浪漫星空灯投影灯小夜灯彩灯儿童礼物送女孩男孩送孩子生日礼品生日灯 生日快乐+精美包装	https://try.jd.com/704294.html	申请成功！
--------------------
--------------------
Number 233
Offered 5	Applied 856	Price 48.00	State 活动已开始，请快快申请吧！
No need follow
JS ret = 0
Applied：白酒 永丰二锅头 永丰牌北京二锅头金刚荣耀系列42度清香型白酒 黄瓶单瓶装	https://try.jd.com/708041.html	申请成功！
--------------------
--------------------
Number 234
Offered 1	Applied 528	Price 46.90	State 活动已开始，请快快申请吧！
JS ret = 0
Applied：芋圆鲜芋仙芋圆甜品组合2斤装芋圆甜品原料台湾手工芋圆糖水材料奶茶配料清汤三口味七夕情人节礼物	https://try.jd.com/706458.html	申请成功！
--------------------
------------------

In [ ]:
browser.close()

In [ ]:
#follow_link = follow_div.find_element_by_xpath(".//a[1]")
#follow_link = follow_div.find_element_by_class_name('x')
for i in allCookies:
    print(i['name']+"\t"+i['value'])
print(len(allCookies))

In [7]:
js1 = r'''
 try {
	window.fingerprint = {},
	function t() {
		fingerprint.config = {
			fpb_send_data: 'body={"appname": "jdwebm_hf","jdkey": "","whwswswws": "","businness": "","body": {}}',
			api: {
				canvas_spendtime: 0
			}
		},
		fingerprint.broswer = {
			getNavigatorPlatform: function () {
				return navigator.platform ? navigator.platform : "unknown"
			},
			getDeviceMemory: function () {
				return navigator.deviceMemory ? navigator.deviceMemory : 0
			},
			rB: function () {
				var e = [],
				t = ["__webdriver_evaluate", "__selenium_evaluate", "__webdriver_script_function", "__webdriver_script_func", "__webdriver_script_fn", "__fxdriver_evaluate", "__driver_unwrapped", "__webdriver_unwrapped", "__driver_evaluate", "__selenium_unwrapped", "__fxdriver_unwrapped"],
				r = ["_phantom", "__nightmare", "_selenium", "callPhantom", "callSelenium", "_Selenium_IDE_Recorder"];
				for (var n in r) {
					var i = r[n];
					window[i] && e.push({
						key: "window",
						value: i
					})
				}
				for (var o in t) {
					r = t[o];
					window.document[r] && e.push({
						key: "window_document",
						value: r
					})
				}
				for (var a in window.document)
					a.match(/\$[a-z]dc_/) && window.document[a].cache_ && e.push({
						key: "document",
						value: a
					});
				return window.external && JSON.stringify(window.external) && -1 != JSON.stringify(window.external).indexOf("Sequentum") && e.push({
					key: "window_external",
					value: 1
				}),
				window.document.documentElement.getAttribute("selenium") && e.push({
					key: "document_selenium",
					value: 1
				}),
				window.document.documentElement.getAttribute("webdriver") && e.push({
					key: "document_webdriver",
					value: 1
				}),
				window.document.documentElement.getAttribute("driver") && e.push({
					key: "document_driver",
					value: 1
				}),
				e
			},
			getHeadless: function () {
				return this.rB().length > 0 ? 1 : navigator.userAgent.toLocaleLowerCase().indexOf("headless") > -1 ? 1 : 0
			},
			getLocation: function () {
				return fingerprint.util.MD5.hex_md5(location.href.split("?")[0])
			},
			getUserAgent: function () {
				return fingerprint.util.MD5.hex_md5(navigator.userAgent)
			},
			getCanvas: function () {
				try {
					var e = new Date,
					t = [],
					r = document.createElement("canvas");
					r.width = 2e3,
					r.height = 200,
					r.style.display = "inline";
					var n = r.getContext("2d");
					return n.rect(0, 0, 10, 10),
					n.rect(2, 2, 6, 6),
					t.push("canvas winding:" + (!1 === n.isPointInPath(5, 5, "evenodd") ? "yes" : "no")),
					n.textBaseline = "alphabetic",
					n.fillStyle = "#f60",
					n.fillRect(125, 1, 62, 20),
					n.fillStyle = "#069",
					n.font = "11pt no-real-font-123",
					n.fillText("Cwm fjordbank glyphs vext quiz, 😃", 2, 15),
					n.fillStyle = "rgba(102, 204, 0, 0.2)",
					n.font = "18pt Arial",
					n.fillText("Cwm fjordbank glyphs vext quiz, 😃", 4, 45),
					n.globalCompositeOperation = "multiply",
					n.fillStyle = "rgb(255,0,255)",
					n.beginPath(),
					n.arc(50, 50, 50, 0, 2 * Math.PI, !0),
					n.closePath(),
					n.fill(),
					n.fillStyle = "rgb(0,255,255)",
					n.beginPath(),
					n.arc(100, 50, 50, 0, 2 * Math.PI, !0),
					n.closePath(),
					n.fill(),
					n.fillStyle = "rgb(255,255,0)",
					n.beginPath(),
					n.arc(75, 100, 50, 0, 2 * Math.PI, !0),
					n.closePath(),
					n.fill(),
					n.fillStyle = "rgb(255,0,255)",
					n.arc(75, 75, 75, 0, 2 * Math.PI, !0),
					n.arc(75, 75, 25, 0, 2 * Math.PI, !0),
					n.fill("evenodd"),
					t.push("canvas fp:" + fingerprint.util.MD5.hex_md5(r.toDataURL())),
					fingerprint.config.api.canvas_spendtime = (new Date).getTime() - e.getTime(),
					t.join("~")
				} catch (e) {
					return console.log(e),
					""
				}
			},
			getPluginName: function () {
				var e = "";
				if (navigator.plugins.length > 0)
					for (i = 0; i < navigator.plugins.length; i++)
						e += navigator.plugins[i].name + ";";
				return e
			},
			getPluginNum: function () {
				return navigator.plugins.length
			},
			getScreenResolution: function () {
				return window.screen.width + "*" + window.screen.height
			}
		},
		fingerprint.util = {
			G: "",
			getCookie: function (e) {
				var t,
				r = new RegExp("(^| )" + e + "=([^;]*)(;|$)");
				return (t = document.cookie.match(r)) ? unescape(t[2]) : ""
			},
			getStorage: function (e) {
				if (window.Storage && window.localStorage && window.localStorage instanceof Storage)
					return JSON.parse(localStorage.getItem(e))
			},
			setStorage: function (e, t) {
				window.Storage && window.localStorage && window.localStorage instanceof Storage && localStorage.setItem(e, JSON.stringify(t))
			},
			getFpb: function (e) {
				var t,
				r = "";
				if (window.navigator.cookieEnabled) {
					var n = window.document.cookie.indexOf(e + "=");
					if (-1 != n) {
						var i = e.length + 1;
						n += i;
						var o = window.document.cookie.indexOf(";", n);
						-1 == o && (o = window.document.cookie.length);
						try {
							t = decodeURIComponent(window.document.cookie.substring(n, o)) || ""
						} catch (e) {
							t = window.document.cookie.substring(n, o) || ""
						}
						r || (r = t)
					}
				}
				try {
					window.localStorage && (t = window.localStorage.getItem(e) || "", r || (r = t))
				} catch (e) {}
				try {
					window.sessionStorage && (t = window.sessionStorage.getItem(e) || "", r || (r = t))
				} catch (e) {}
				return t = window.name,
				r || (r = t),
				r || (r = t),
				r || ""
			},
			getDateFormat: function (e, t) {
				var r = {
					"M+": e.getMonth() + 1,
					"d+": e.getDate(),
					"h+": e.getHours(),
					"m+": e.getMinutes(),
					"s+": e.getSeconds(),
					"q+": Math.floor((e.getMonth() + 3) / 3),
					S: e.getMilliseconds()
				};
				/(y+)/.test(t) && (t = t.replace(RegExp.$1, (e.getFullYear() + "").substr(4 - RegExp.$1.length)));
				for (var n in r)
					new RegExp("(" + n + ")").test(t) && (t = t.replace(RegExp.$1, 1 == RegExp.$1.length ? r[n] : ("00" + r[n]).substr(("" + r[n]).length)));
				return t
			},
			MD5: {
				chrsz: 8,
				G: "",
				hex_md5: function (e) {
					return this.binl2hex(this.core_md5(this.str2binl(e), e.length * this.chrsz))
				},
				core_md5: function (e, t) {
					e[t >> 5] |= 128 << t % 32,
					e[14 + (t + 64 >>> 9 << 4)] = t;
					for (var r = 1732584193, n = -271733879, i = -1732584194, o = 271733878, a = 0; a < e.length; a += 16) {
						var s = r,
						d = n,
						c = i,
						u = o;
						r = this.md5_ff(r, n, i, o, e[a + 0], 7, -680876936),
						o = this.md5_ff(o, r, n, i, e[a + 1], 12, -389564586),
						i = this.md5_ff(i, o, r, n, e[a + 2], 17, 606105819),
						n = this.md5_ff(n, i, o, r, e[a + 3], 22, -1044525330),
						r = this.md5_ff(r, n, i, o, e[a + 4], 7, -176418897),
						o = this.md5_ff(o, r, n, i, e[a + 5], 12, 1200080426),
						i = this.md5_ff(i, o, r, n, e[a + 6], 17, -1473231341),
						n = this.md5_ff(n, i, o, r, e[a + 7], 22, -45705983),
						r = this.md5_ff(r, n, i, o, e[a + 8], 7, 1770035416),
						o = this.md5_ff(o, r, n, i, e[a + 9], 12, -1958414417),
						i = this.md5_ff(i, o, r, n, e[a + 10], 17, -42063),
						n = this.md5_ff(n, i, o, r, e[a + 11], 22, -1990404162),
						r = this.md5_ff(r, n, i, o, e[a + 12], 7, 1804603682),
						o = this.md5_ff(o, r, n, i, e[a + 13], 12, -40341101),
						i = this.md5_ff(i, o, r, n, e[a + 14], 17, -1502002290),
						n = this.md5_ff(n, i, o, r, e[a + 15], 22, 1236535329),
						r = this.md5_gg(r, n, i, o, e[a + 1], 5, -165796510),
						o = this.md5_gg(o, r, n, i, e[a + 6], 9, -1069501632),
						i = this.md5_gg(i, o, r, n, e[a + 11], 14, 643717713),
						n = this.md5_gg(n, i, o, r, e[a + 0], 20, -373897302),
						r = this.md5_gg(r, n, i, o, e[a + 5], 5, -701558691),
						o = this.md5_gg(o, r, n, i, e[a + 10], 9, 38016083),
						i = this.md5_gg(i, o, r, n, e[a + 15], 14, -660478335),
						n = this.md5_gg(n, i, o, r, e[a + 4], 20, -405537848),
						r = this.md5_gg(r, n, i, o, e[a + 9], 5, 568446438),
						o = this.md5_gg(o, r, n, i, e[a + 14], 9, -1019803690),
						i = this.md5_gg(i, o, r, n, e[a + 3], 14, -187363961),
						n = this.md5_gg(n, i, o, r, e[a + 8], 20, 1163531501),
						r = this.md5_gg(r, n, i, o, e[a + 13], 5, -1444681467),
						o = this.md5_gg(o, r, n, i, e[a + 2], 9, -51403784),
						i = this.md5_gg(i, o, r, n, e[a + 7], 14, 1735328473),
						n = this.md5_gg(n, i, o, r, e[a + 12], 20, -1926607734),
						r = this.md5_hh(r, n, i, o, e[a + 5], 4, -378558),
						o = this.md5_hh(o, r, n, i, e[a + 8], 11, -2022574463),
						i = this.md5_hh(i, o, r, n, e[a + 11], 16, 1839030562),
						n = this.md5_hh(n, i, o, r, e[a + 14], 23, -35309556),
						r = this.md5_hh(r, n, i, o, e[a + 1], 4, -1530992060),
						o = this.md5_hh(o, r, n, i, e[a + 4], 11, 1272893353),
						i = this.md5_hh(i, o, r, n, e[a + 7], 16, -155497632),
						n = this.md5_hh(n, i, o, r, e[a + 10], 23, -1094730640),
						r = this.md5_hh(r, n, i, o, e[a + 13], 4, 681279174),
						o = this.md5_hh(o, r, n, i, e[a + 0], 11, -358537222),
						i = this.md5_hh(i, o, r, n, e[a + 3], 16, -722521979),
						n = this.md5_hh(n, i, o, r, e[a + 6], 23, 76029189),
						r = this.md5_hh(r, n, i, o, e[a + 9], 4, -640364487),
						o = this.md5_hh(o, r, n, i, e[a + 12], 11, -421815835),
						i = this.md5_hh(i, o, r, n, e[a + 15], 16, 530742520),
						n = this.md5_hh(n, i, o, r, e[a + 2], 23, -995338651),
						r = this.md5_ii(r, n, i, o, e[a + 0], 6, -198630844),
						o = this.md5_ii(o, r, n, i, e[a + 7], 10, 1126891415),
						i = this.md5_ii(i, o, r, n, e[a + 14], 15, -1416354905),
						n = this.md5_ii(n, i, o, r, e[a + 5], 21, -57434055),
						r = this.md5_ii(r, n, i, o, e[a + 12], 6, 1700485571),
						o = this.md5_ii(o, r, n, i, e[a + 3], 10, -1894986606),
						i = this.md5_ii(i, o, r, n, e[a + 10], 15, -1051523),
						n = this.md5_ii(n, i, o, r, e[a + 1], 21, -2054922799),
						r = this.md5_ii(r, n, i, o, e[a + 8], 6, 1873313359),
						o = this.md5_ii(o, r, n, i, e[a + 15], 10, -30611744),
						i = this.md5_ii(i, o, r, n, e[a + 6], 15, -1560198380),
						n = this.md5_ii(n, i, o, r, e[a + 13], 21, 1309151649),
						r = this.md5_ii(r, n, i, o, e[a + 4], 6, -145523070),
						o = this.md5_ii(o, r, n, i, e[a + 11], 10, -1120210379),
						i = this.md5_ii(i, o, r, n, e[a + 2], 15, 718787259),
						n = this.md5_ii(n, i, o, r, e[a + 9], 21, -343485551),
						r = this.safe_add(r, s),
						n = this.safe_add(n, d),
						i = this.safe_add(i, c),
						o = this.safe_add(o, u)
					}
					return Array(r, n, i, o)
				},
				md5_cmn: function (e, t, r, n, i, o) {
					return this.safe_add(this.bit_rol(this.safe_add(this.safe_add(t, e), this.safe_add(n, o)), i), r)
				},
				md5_ff: function (e, t, r, n, i, o, a) {
					return this.md5_cmn(t & r | ~t & n, e, t, i, o, a)
				},
				md5_gg: function (e, t, r, n, i, o, a) {
					return this.md5_cmn(t & n | r & ~n, e, t, i, o, a)
				},
				md5_hh: function (e, t, r, n, i, o, a) {
					return this.md5_cmn(t ^ r ^ n, e, t, i, o, a)
				},
				md5_ii: function (e, t, r, n, i, o, a) {
					return this.md5_cmn(r ^ (t | ~n), e, t, i, o, a)
				},
				safe_add: function (e, t) {
					var r = (65535 & e) + (65535 & t);
					return (e >> 16) + (t >> 16) + (r >> 16) << 16 | 65535 & r
				},
				bit_rol: function (e, t) {
					return e << t | e >>> 32 - t
				},
				str2binl: function (e) {
					for (var t = Array(), r = (1 << this.chrsz) - 1, n = 0; n < e.length * this.chrsz; n += this.chrsz)
						t[n >> 5] |= (e.charCodeAt(n / this.chrsz) & r) << n % 32;
					return t
				},
				binl2hex: function (e) {
					for (var t = "0123456789abcdef", r = "", n = 0; n < 4 * e.length; n++)
						r += t.charAt(e[n >> 2] >> n % 4 * 8 + 4 & 15) + t.charAt(e[n >> 2] >> n % 4 * 8 & 15);
					return r
				}
			},
			Base64: {
				_keyStr: "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/=",
				encode: function (e) {
					var t,
					r,
					n,
					i,
					o,
					a,
					s,
					d = "",
					c = 0;
					for (e = this._utf8_encode(e); c < e.length; )
						i = (t = e.charCodeAt(c++)) >> 2, o = (3 & t) << 4 | (r = e.charCodeAt(c++)) >> 4, a = (15 & r) << 2 | (n = e.charCodeAt(c++)) >> 6, s = 63 & n, isNaN(r) ? a = s = 64 : isNaN(n) && (s = 64), d = d + this._keyStr.charAt(i) + this._keyStr.charAt(o) + this._keyStr.charAt(a) + this._keyStr.charAt(s);
					return d
				},
				_utf8_encode: function (e) {
					e = e.replace(/\r\n/g, "\n");
					for (var t = "", r = 0; r < e.length; r++) {
						var n = e.charCodeAt(r);
						n < 128 ? t += String.fromCharCode(n) : n > 127 && n < 2048 ? (t += String.fromCharCode(n >> 6 | 192), t += String.fromCharCode(63 & n | 128)) : (t += String.fromCharCode(n >> 12 | 224), t += String.fromCharCode(n >> 6 & 63 | 128), t += String.fromCharCode(63 & n | 128))
					}
					return t
				}
			}
		},
		getFingerprint = function () {
			try {
				return "v001" + fingerprint.util.Base64.encode(JSON.stringify({
						b: fingerprint.util.getCookie("shshshfpa"),
						c: fingerprint.util.getFpb("shshshfpb"),
						d: fingerprint.broswer.getNavigatorPlatform(),
						f: fingerprint.broswer.getDeviceMemory(),
						g: fingerprint.broswer.getHeadless(),
						h: fingerprint.broswer.getCanvas(),
						i: fingerprint.config.api.canvas_spendtime,
						j: fingerprint.util.getDateFormat(new Date, "yyyy-MM-dd hh:mm:ss"),
						k: fingerprint.broswer.getLocation(),
						l: fingerprint.broswer.getUserAgent(),
						m: fingerprint.util.getCookie("pin") || fingerprint.util.getCookie("pt_pin"),
						n: fingerprint.broswer.getPluginName(),
						o: fingerprint.broswer.getPluginNum(),
						p: fingerprint.broswer.getScreenResolution()
					}))
			} catch (t) {
				return console.log(t),
				"error :" + t
			}
		}
	}
	()
} catch (e) {
	getFingerprint = function () {
		return console.log(e),
		"error :" + e
	}
}
var ret = fingerprint.broswer.getHeadless();
return ret;
'''

In [ ]:
loggif='https://mercury.jd.com/log.gif?t=www.100000&m=UA-J2011-1&pin=Sanord&uid=1553650534805631944147&sid=1553650534805631944147|1&v=je%3D0%24sc%3D24-bit%24sr%3D1536x864%24ul%3Den-us%24cs%3DUTF-8%24dt%3D%E8%AF%95%E7%94%A8%E8%AF%A6%E6%83%85%24hn%3Dtry.jd.com%24fl%3D-%24os%3Dwin%24br%3Dchrome%24bv%3D72.0.3626.121%24wb%3D1553650535%24xb%3D1553650535%24yb%3D1553650535%24zb%3D1%24cb%3D9%24usc%3Ddirect%24ucp%3D-%24umd%3Dnone%24uct%3D-%24lt%3D0%24ct%3D1553651129415%24tad%3D-%24mba_finger%3Dv001eyJiIjoiIiwiYyI6IiYzQUI5RDIzRjdBNEIzQzlCPVVaTjNZQlJXR1ZTSEJNWEs2VlZHNEJNNldSSlREUUw2UDVJNVgyUVhSMlZUTzZWV09CSzZDU0hBS05UM0VSRFoySkc0R0M0NkdVNFY0NDNSQVBKTUhTSjZHTSIsImQiOiJXaW4zMiIsImYiOjgsImciOjAsImgiOiJjYW52YXMgd2luZGluZzp5ZXN%2BY2FudmFzIGZwOmNhYTRhMWVlZjkyOTlmYWE4OWM0ZTFlOTc2ZGY4NTA5IiwiaSI6ODQsImoiOiIyMDE5LTAzLTI3IDA5OjQ1OjI5IiwiayI6IjFkZjhkMTIzY2RiYzZhMmMxYzhlY2Y4ZDViNDkyNTc3IiwibCI6Ijc1YjMzNDFkYTllNzIwOGZjMDNkMDkwOWY2OTk5MWFhIiwibSI6IlNhbm9yZCIsIm4iOiJDaHJvbWUgUERGIFBsdWdpbjtDaHJvbWUgUERGIFZpZXdlcjtOYXRpdmUgQ2xpZW50OyIsIm8iOjMsInAiOiIxNTM2Kjg2NCJ9%24fpftime%3D113%24pinid%3DGmLF-XtUvGs%24jdv%3D122270672%7Cdirect%7C-%7Cnone%7C-%7C1553650534808%24dataver%3D0.1&ref=&rm=1553651129560'
import urllib 
rawurl=loggif 
url=urllib.parse.unquote(rawurl)
print(url)

In [ ]:
text = u'活动已开始，请快快申请吧！'
text.find(u'已提交')

In [ ]:
filter_in = re.compile(u'(杯|酒|壶|茶|玉|玛瑙|灯|桌|咖啡|笔记本|塑料袋|[酱|调]料|文玩|水培|笔|[U|优]盘|显示器.*支架|摄像头|乐器|背包|冰箱|手环|风扇|电视|玩具|书籍|办公用品|得力|洗衣机|净化器|螺丝刀|机器人|伞|美的|阿道夫)')
i = 1
for name,url in try_items.items():
    if filter_in.search(name)!=None:
        print(name)
        print(filter_in.search(name))

In [ ]:
try_items

In [ ]:
filter_in.search('驰爵 小米9/小米9SE手机壳玻璃小米9透明尊享版镜面保护套全包硅胶防摔创意卡通男女款时尚潮牌外壳 小米9/米9尊享版【黑白阴阳鱼】送钢化膜')

In [ ]:
check_url = 'https://mercury.jd.com/log.gif?t=magic.000001&m=UA-J2011-1&pin=Sanord&uid=1562811976815118280586&sid=1562811976815118280586|1&v=je%3D0%24sc%3D24-bit%24sr%3D1536x864%24ul%3Den-us%24cs%3DUTF-8%24dt%3D%E8%AF%95%E7%94%A8%E8%AF%A6%E6%83%85%24hn%3Dtry.jd.com%24fl%3D-%24os%3Dwin%24br%3Dchrome%24bv%3D72.0.3626.121%24wb%3D1562811977%24xb%3D1562811977%24yb%3D1562811977%24zb%3D1%24cb%3D9%24usc%3Ddirect%24ucp%3D-%24umd%3Dnone%24uct%3D-%24ct%3D1562813331884%24lt%3D0%24tad%3D-%24t1%3Dmagictree%24t2%3DX%24p0%3D%25252F%25252F*%25255B%252540id%25253D%252522product-intro%252522%25255D%25252Fdiv%25255B2%25255D%25252Fdiv%25255B4%25255D%25252Fdiv%25252Fdiv%25252Fdiv%25255B4%25255D%25252Fa%257C%257C%257C%25252F%25252F*%25255B%252540id%25253D%252522product-intro%252522%25255D%25252Fdiv%25255B2%25255D%25252Fdiv%25255B4%25255D%25252Fdiv%25252Fdiv%25252Fdiv%25255B4%25255D%257C%257C%257C%25252Fhtml%25252Fbody%25252Fdiv%25255B4%25255D%25252Fdiv%25255B2%25255D%25252Fdiv%25252Fdiv%25255B2%25255D%25252Fdiv%25255B4%25255D%25252Fdiv%25252Fdiv%25252Fdiv%25255B4%25255D%25252F%257C%257C%257C12%257C%257C%257C553x525%257C%257C%257C1212x2111%257C%257C%257C0%257C%257C%257Ctry%257C%257C%257CSQAN-1%257C%257C%257Chttps%25253A%25252F%25252Ftry.jd.com%25252F677721.html%252523this%257C%257C%257C%2525E7%252594%2525B3%2525E8%2525AF%2525B7%2525E8%2525AF%252595%2525E7%252594%2525A8%257C%257C%257C-%24pinid%3DGmLF-XtUvGs%24jdv%3D122270672%7Cdirect%7C-%7Cnone%7C-%7C1562811976820%24dataver%3D0.1&ref=&rm=1562813331890'
check_url.split('&')

In [ ]:
p0='%2F%2F*%5B%40id%3D%22product-intro%22%5D%2Fdiv%5B2%5D%2Fdiv%5B4%5D%2Fdiv%2Fdiv%2Fdiv%5B4%5D%2Fa|||%2F%2F*%5B%40id%3D%22product-intro%22%5D%2Fdiv%5B2%5D%2Fdiv%5B4%5D%2Fdiv%2Fdiv%2Fdiv%5B4%5D|||%2Fhtml%2Fbody%2Fdiv%5B4%5D%2Fdiv%5B2%5D%2Fdiv%2Fdiv%5B2%5D%2Fdiv%5B4%5D%2Fdiv%2Fdiv%2Fdiv%5B4%5D%2F|||12|||553x525|||1212x2111|||0|||try|||SQAN-1|||https%3A%2F%2Ftry.jd.com%2F677721.html%23this|||%E7%94%B3%E8%AF%B7%E8%AF%95%E7%94%A8|||-'
urllib.parse.unquote(p0)

In [ ]:
print(2**8)

In [ ]:
print(6/7.0)